# Коллаборативная фильтрация

ПАКЕТ SURPRISE

используйте данные MovieLens 1M

можно использовать любые модели из пакета

получите RMSE на тестовом сете 0.87 и ниже

Комментарий преподавателя :
В ДЗ на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе CrossValidation (5 фолдов), а не отложенном датасете.

In [2]:
!pip3 install surprise

  Using cached surprise-0.1-py2.py3-none-any.whl (1.8 kB)
  Using cached scikit-surprise-1.1.1.tar.gz (11.8 MB)
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp38-cp38-win_amd64.whl size=728748 sha256=3914541033746d3fe5401ddc7b1a8ca379cab74cefd9006bb53292f7366d3a9d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\20\91\57\2965d4cff1b8ac7ed1b6fa25741882af3974b54a31759e10b6
Successfully built scikit-surprise


In [3]:
from surprise import KNNWithMeans, KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import pandas as pd

In [7]:
m = ['movie_id', 'title', 'genres']
r = ['user_id', 'movie_id', 'rating', 'timestamp']
movies = pd.read_table('movies.dat', sep='::', header=None, names=m)
ratings = pd.read_table('ratings.dat', sep='::', header=None, names=r)

In [8]:
movies_with_ratings = movies.join(ratings.set_index('movie_id'), on='movie_id').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [9]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.user_id,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),5.0
1,6.0,Toy Story (1995),4.0
2,8.0,Toy Story (1995),4.0
3,9.0,Toy Story (1995),5.0
4,10.0,Toy Story (1995),5.0


In [10]:
ratings.rating.min()

1

In [11]:
ratings.rating.max()

5

In [12]:
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [13]:
trainset, testset = train_test_split(data, test_size=.15)

In [14]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [15]:
test_pred = algo.test(testset)

In [16]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8834


0.8834058275860072

In [17]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [18]:
test_pred = algo.test(testset)

In [19]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8533


0.8532816284641658

Условиям задачи удовлетворяет, но попробуем иные модели

In [20]:
algo = KNNBasic(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [21]:
test_pred = algo.test(testset)

In [22]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9194


0.9193926263831509

In [23]:
from surprise import SVDpp

In [25]:
algo = SVDpp()
algo.fit(trainset)

In [26]:
test_pred = algo.test(testset)

In [27]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8559


0.8558946109110145

In [28]:
from surprise import SlopeOne

In [29]:
algo = SlopeOne()
algo.fit(trainset)

<ipython-input-29-e07553bd1040>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  algo.fit(trainset)


In [30]:
test_pred = algo.test(testset)

In [31]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9031


0.9031351921980886

In [32]:
from surprise import CoClustering

In [33]:
algo = CoClustering()
algo.fit(trainset)

<ipython-input-33-b00c277ee28c>:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  algo.fit(trainset)


In [34]:
test_pred = algo.test(testset)

In [35]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9123


0.9122744292722001

Как вывод, самое лучшее значение показала первая модель. Правда я не заморачивался с параметрами.